In [2]:
import pandas as pd
import numpy as np

def buy_signal1(prices, drop_threshold=-0.005, stabilization_threshold=0.001, drop_lookback=7, stabilization_lookback=1):
    if isinstance(prices, list):
        prices = pd.Series(prices)

    if len(prices) < drop_lookback + stabilization_lookback:
        return False

    price_changes = prices.pct_change().dropna()
    recent_drops = price_changes.iloc[-(drop_lookback + stabilization_lookback):-stabilization_lookback] < drop_threshold
    recent_stabilizations = price_changes.iloc[-stabilization_lookback:] > stabilization_threshold
    return recent_drops.any() and recent_stabilizations.any()

def buy_signal2(prices, drop_threshold=-0.005, stabilization_period=1):
    if len(prices) < 15 + stabilization_period:
        return False
    significant_drop = prices.pct_change(periods=15).iloc[-stabilization_period - 1] <= drop_threshold
    stabilization_prices = prices.iloc[-stabilization_period:]
    stabilized = stabilization_prices.pct_change().dropna().abs().max() <= 0.002
    return significant_drop and stabilized

def sell_signal1(prices, purchase_price, threshold=0.005, stop_loss=0.02):
    lookback_period = 6
    required_rises = 4
    downward_comparisons = 2

    if len(prices) < lookback_period + 1:
        return False

    current_price = prices.iloc[-1]
    peak_price = prices.iloc[-(lookback_period + 1):-downward_comparisons].max()
    relative_increase = (peak_price - prices[0]) / prices[0]

    is_stop_loss_triggered = (purchase_price - current_price) / purchase_price >= stop_loss
    rises = (prices.diff().iloc[-(lookback_period + 1):-downward_comparisons] > 0).sum()
    recent_downward_trend = (prices.diff().iloc[-downward_comparisons:] < 0).all()

    if is_stop_loss_triggered or (relative_increase >= threshold and rises >= required_rises and
        recent_downward_trend and current_price > purchase_price):
        return True

    return False

def sell_signal2(prices, rise_threshold=0.008, stabilization_period=4):
    if len(prices) < 10 + stabilization_period:
        return False
    significant_rise = prices.pct_change(periods=10).iloc[-stabilization_period - 1] >= rise_threshold
    stabilization_prices = prices.iloc[-stabilization_period:]
    stabilized = stabilization_prices.pct_change().dropna().abs().max() <= 0.001
    return significant_rise and stabilized

def optimize_parameters(data):
    best_profit = -np.inf
    best_params = None
    # Define a grid of parameters to search
    param_grid = {
        'drop_threshold': [-0.01, -0.005, -0.003],
        'stabilization_threshold': [0.001, 0.002],
        'drop_lookback': [5, 7, 10],
        'stabilization_lookback': [1, 2],
        'threshold': [0.005, 0.01],
        'stop_loss': [0.01, 0.02, 0.03],
        'rise_threshold': [0.005, 0.008, 0.01],
        'stabilization_period': [2, 4, 6]
    }

    for dt in param_grid['drop_threshold']:
        for st in param_grid['stabilization_threshold']:
            for dl in param_grid['drop_lookback']:
                for sl in param_grid['stabilization_lookback']:
                    for th in param_grid['threshold']:
                        for sloss in param_grid['stop_loss']:
                            for rt in param_grid['rise_threshold']:
                                for sp in param_grid['stabilization_period']:
                                    profit = simulate_trading_strategy(data, dt, st, dl, sl, th, sloss, rt, sp)
                                    if profit > best_profit:
                                        best_profit = profit
                                        best_params = (dt, st, dl, sl, th, sloss, rt, sp)
    return best_params, best_profit

def simulate_trading_strategy(data, drop_threshold, stabilization_threshold, drop_lookback, stabilization_lookback, threshold, stop_loss, rise_threshold, stabilization_period):
    capital = 10000  # Starting capital
    position = 0  # Current position in stocks
    purchase_price = 0

    for i in range(max(drop_lookback, stabilization_lookback, stabilization_period) + 1, len(data)):
        prices = data['Close'].iloc[:i]

        if position == 0:  # If not holding stock
            if buy_signal1(prices, drop_threshold, stabilization_threshold, drop_lookback, stabilization_lookback) or buy_signal2(prices, drop_threshold, stabilization_period):
                position = capital // prices.iloc[-1]
                purchase_price = prices.iloc[-1]
                capital -= position * purchase_price

        else:  # If holding stock
            if sell_signal1(prices, purchase_price, threshold, stop_loss) or sell_signal2(prices, rise_threshold, stabilization_period):
                capital += position * prices.iloc[-1]
                position = 0

    # Sell any remaining stocks at the end
    if position > 0:
        capital += position * data['Close'].iloc[-1]

    return capital - 10000

# Assume df is the DataFrame with your price data
df = pd.read_csv('JSWENERGY_NSE_1min_data.csv', parse_dates=['DateTime'], index_col='DateTime')

best_params, best_profit = optimize_parameters(df)
print(f'Best Parameters: {best_params}')
print(f'Best Profit: {best_profit}')


FileNotFoundError: [Errno 2] No such file or directory: 'JSWENERGY_NSE_1min_data.csv'